# Setup notebook to communicate with server and push information into Kafka Topic

Welcome, please run the following lines of code to connect to the kafka server, make sure to store the .csv file in the same folder where this notebooks is. Have fun and good luck!

In [4]:
# pip install kafka

Note: you may need to restart the kernel to use updated packages.


In [1]:
# pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


#### Libraries

In [2]:
import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
from json import dumps
from tqdm import tqdm

#### Connect to server and to topic to consume

In [3]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
if producer.bootstrap_connected():
    print(f"Successfully connected to bootstrap server")
else:
    print("Couldn't connect to bootstrap server.")

TOPIC_NAME = "ml-raw-dns"

Successfully connected to bootstrap server


#### Define the function to push the dataset 

In [4]:
def produce_message(producer_instance, topic, message):
    producer_instance.send(topic, message)
    producer_instance.flush()
    return

#### Push the information

In [5]:
with open("C:/Users/Welcome/OneDrive/Desktop/uottawa/Second Term/AI foe CS/Assignments/A3/Env/data/Static_dataset.csv") as f:
    start_time = datetime.now()
    for i, line in tqdm(enumerate(f)):
        #print(i, line)
        produce_message(producer_instance=producer, topic=TOPIC_NAME, message=line)
    end_time = datetime.now()
    print(f"Batch took {end_time-start_time} time for ingesting data")

print("Ingestion Completed")

0it [00:00, ?it/s]

268075it [06:10, 722.65it/s]

Batch took 0:06:10.975208 time for ingesting data
Ingestion Completed


In [6]:
def consume_message(consumer_instance, topic, num_messages):
    messages = []
    for i in range(num_messages):
        msg = consumer_instance.poll(timeout_ms=1000)
        messages.append(msg)
    return messages

from kafka import KafkaConsumer
from json import loads

TOPIC_NAME = "ml-raw-dns"
consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-group',
    value_deserializer=lambda x: loads(x.decode('utf-8')))

if consumer.bootstrap_connected():
    print(f"Successfully connected to bootstrap server")
else:
    print("Couldn't connect to bootstrap server.")
start_time = datetime.now()
messages = consume_message(consumer_instance=consumer, topic=TOPIC_NAME, num_messages=1000)
end_time = datetime.now()
print(f"Batch took {end_time-start_time} time for consuming data")
#print the first message
print(messages[0])

Successfully connected to bootstrap server
Batch took 0:07:29.840421 time for consuming data
{TopicPartition(topic='ml-raw-dns', partition=0): [ConsumerRecord(topic='ml-raw-dns', partition=0, offset=0, timestamp=1699908303034, timestamp_type=0, key=None, value='ï»¿timestamp,FQDN_count,subdomain_length,upper,lower,numeric,entropy,special,labels,labels_max,labels_average,longest_word,sld,len,subdomain,Target Attack\n', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=173, serialized_header_size=-1), ConsumerRecord(topic='ml-raw-dns', partition=0, offset=1, timestamp=1699908303151, timestamp_type=0, key=None, value='56:19.8,27,10,0,10,11,2.57041707,6,6,7,3.666666667,2,192,14,1,1\n', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=67, serialized_header_size=-1), ConsumerRecord(topic='ml-raw-dns', partition=0, offset=2, timestamp=1699908303151, timestamp_type=0, key=None, value='07:23.9,27,10,0,10,11,2.767194749,6,6,7,3.666666667,2,192,14,1,1